<a href="https://colab.research.google.com/github/RajuMekala01/ev_project/blob/main/EV_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
data=pd.read_csv('/content/Electric_Vehicle_Population_Data.csv.zip')
data.shape



(124716, 17)